In [6]:
import numpy as np
from PIL import Image

def singular_values_for_channel(channel):
    # channel: 2D numpy array (float)
    # zwraca wartości singularne (sigma) i macierze U, Vt
    U, s, Vt = np.linalg.svd(channel, full_matrices=False)
    return s, U, Vt

def find_k_for_energy(s, threshold=0.90):
    energies = s**2
    total = energies.sum()
    cumulative = np.cumsum(energies)
    k = np.searchsorted(cumulative, threshold * total) + 1
    return k

def reconstruct_from_svd(U, s, Vt, k):
    U_k = U[:, :k]
    s_k = s[:k]
    Vt_k = Vt[:k, :]
    return (U_k * s_k) @ Vt_k

def process_image_find_k(path, threshold=0.90):
    im = Image.open(path).convert("RGB")
    arr = np.array(im, dtype=float)
    ks = []
    recon_channels = []

    for c in range(3):
        channel = arr[:, :, c]
        s, U, Vt = singular_values_for_channel(channel)
        k = find_k_for_energy(s, threshold)
        ks.append(k)
        recon = reconstruct_from_svd(U, s, Vt, k)
        recon_channels.append(recon)

    recon_img = np.stack(recon_channels, axis=2)
    recon_img = np.clip(recon_img, 0, 255).astype(np.uint8)
    recon_pil = Image.fromarray(recon_img)

    output_path = r"12_rec.png"
    recon_pil.save(output_path)

    print(f"Rekonstrukcja zapisana jako: {output_path}")
    print(f"Liczba wartości singularnych (R, G, B): {ks}")
    print(f"Maksymalne k (dla całego obrazu): {max(ks)}")

    return ks, output_path


if __name__ == "__main__":
    process_image_find_k("12.webp", threshold=0.90)


Rekonstrukcja zapisana jako: 12_rec.png
Liczba wartości singularnych (R, G, B): [np.int64(1), np.int64(1), np.int64(1)]
Maksymalne k (dla całego obrazu): 1


In [7]:
#pytorch

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def reconstruct(U, S, Vh, k):
    Uk = U[:, :k]
    Sk = torch.diag(S[:k])
    Vhk = Vh[:k, :]
    return Uk @ Sk @ Vhk

# Główna funkcja
def svd_image_compression(path="12.webp", threshold=0.90):
    img = Image.open(path).convert("RGB")
    arr = np.array(img, dtype=np.float32)
    tensor = torch.from_numpy(arr).to(device)
    H, W, C = tensor.shape

    recon_channels = []
    k_values = []

    for c in range(C):
        channel = tensor[:, :, c]
        U, S, Vh = torch.linalg.svd(channel)
        k = find_k_for_energy(S, threshold)
        k_values.append(k)
        recon = reconstruct(U, S, Vh, k)
        recon_channels.append(recon)

    # Połącz kanały z powrotem
    recon_img = torch.stack(recon_channels, dim=2)
    recon_img = torch.clamp(recon_img, 0, 255).cpu().numpy().astype(np.uint8)

    # Zapisz wynik
    out_path = r"12_torch_rec2.png"
    Image.fromarray(recon_img).save(out_path)

    print(f"Obraz zapisany jako: {out_path}")
    print(f"Wartości k dla kanałów R,G,B: {k_values}")
    print(f"Maksymalne k (wspólny dla całego obrazu): {max(k_values)}")


# Uruchom
if __name__ == "__main__":
    svd_image_compression("12.webp", threshold=0.90)


Obraz zapisany jako: 12_torch_rec2.png
Wartości k dla kanałów R,G,B: [tensor(1), tensor(1), tensor(1)]
Maksymalne k (wspólny dla całego obrazu): 1
